Apply BERT for Text Classification

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')


In [2]:
def load_sample_data():
    """Load inbuilt dataset for text classification"""
    try:
        # Load IMDB movie reviews dataset (sentiment classification)
        print("Loading IMDB dataset...")
        dataset = load_dataset("imdb", split="test[:100]")  # Small subset for demo

        texts = dataset['text']
        labels = dataset['label']  # 0=negative, 1=positive
        label_names = ['negative', 'positive']

        return texts, labels, label_names

    except Exception as e:
        print(f"Could not load IMDB dataset: {e}")
        print("Using sample data instead...")

        # Fallback sample data
        texts = [
            "This movie was absolutely fantastic! Great acting and plot.",
            "Terrible movie, waste of time. Very boring and poorly made.",
            "Amazing film with excellent cinematography and soundtrack.",
            "Not good at all. The story was confusing and characters were weak.",
            "Loved every minute of it! Highly recommend to everyone.",
            "Awful acting and terrible script. Would not watch again.",
            "Outstanding performance by all actors. Brilliant direction.",
            "Very disappointing. Expected much better from this director."
        ]

        labels = [1, 0, 1, 0, 1, 0, 1, 0]  # 1=positive, 0=negative
        label_names = ['negative', 'positive']

        return texts, labels, label_names



In [3]:
def main():
    print("=== BERT Text Classification Demo ===\n")

    # Load data
    texts, true_labels, label_names = load_sample_data()
    print(f"Loaded {len(texts)} samples")

    # Initialize BERT model for sentiment classification
    print("\nLoading pre-trained BERT model...")
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

    try:
        # Create classification pipeline
        classifier = pipeline(
            "sentiment-analysis",
            model=model_name,
            tokenizer=model_name,
            return_all_scores=True
        )
        print("BERT model loaded successfully!")

    except Exception as e:
        print(f"Error loading model: {e}")
        print("Using default sentiment model...")
        classifier = pipeline("sentiment-analysis", return_all_scores=True)

    # Perform classification
    print(f"\n=== Classification Results ===")
    predictions = []

    for i, text in enumerate(texts):
        # Truncate long texts
        text_short = text[:200] + "..." if len(text) > 200 else text

        # Get prediction
        result = classifier(text_short)

        # Extract prediction (assuming binary classification)
        if isinstance(result[0], list):
            scores = result[0]
            pred_label = max(scores, key=lambda x: x['score'])['label']
            confidence = max(scores, key=lambda x: x['score'])['score']
        else:
            pred_label = result[0]['label']
            confidence = result[0]['score']

        # Convert to binary (0/1) if needed
        if pred_label.upper() in ['POSITIVE', 'POS', '1']:
            pred_binary = 1
        else:
            pred_binary = 0

        predictions.append(pred_binary)

        # Display result
        true_label_name = label_names[true_labels[i]]
        pred_label_name = label_names[pred_binary]

        print(f"\n{i+1}. Text: {text_short}")
        print(f"   True: {true_label_name} | Predicted: {pred_label_name} | Confidence: {confidence:.3f}")

        # Show correct/incorrect
        status = "✓" if pred_binary == true_labels[i] else "✗"
        print(f"   Status: {status}")

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predictions)
    print(f"\n=== Performance Metrics ===")
    print(f"Accuracy: {accuracy:.3f}")

    # Detailed report
    print(f"\nClassification Report:")
    print(classification_report(true_labels, predictions, target_names=label_names))

    # Interactive classification
    print(f"\n=== Interactive Classification ===")
    print("Enter text to classify (type 'quit' to exit):")

    while True:
        user_input = input("\nText: ").strip()
        if user_input.lower() in ['quit', 'exit', 'q']:
            break

        if user_input:
            result = classifier(user_input)

            if isinstance(result[0], list):
                scores = result[0]
                for score_dict in scores:
                    label = score_dict['label']
                    score = score_dict['score']
                    print(f"  {label}: {score:.3f}")
            else:
                print(f"  {result[0]['label']}: {result[0]['score']:.3f}")

    print("\nBERT classification demo completed!")

if __name__ == "__main__":
    # Check requirements
    try:
        import transformers
        import datasets
        import sklearn
        print("All packages available!")
        main()
    except ImportError as e:
        print(f"Missing package: {e}")
        print("\nInstall required packages:")
        print("pip install transformers datasets scikit-learn pandas")

        # Minimal fallback
        print("\n=== Minimal Demo (No External Dependencies) ===")

        # Simple keyword-based classifier
        def simple_classify(text):
            positive_words = ['good', 'great', 'excellent', 'amazing', 'fantastic', 'love', 'brilliant']
            negative_words = ['bad', 'terrible', 'awful', 'boring', 'waste', 'disappointing', 'poor']

            text_lower = text.lower()
            pos_count = sum(1 for word in positive_words if word in text_lower)
            neg_count = sum(1 for word in negative_words if word in text_lower)

            if pos_count > neg_count:
                return "positive", pos_count / len(text.split())
            elif neg_count > pos_count:
                return "negative", neg_count / len(text.split())
            else:
                return "neutral", 0.5

        # Test samples
        samples = [
            "This movie was great and amazing!",
            "Terrible and boring waste of time.",
            "The weather is okay today."
        ]

        print("Simple keyword-based classification:")
        for text in samples:
            label, score = simple_classify(text)
            print(f"Text: {text}")
            print(f"Classification: {label} (score: {score:.3f})\n")

All packages available!
=== BERT Text Classification Demo ===

Loading IMDB dataset...


Could not load IMDB dataset: Invalid pattern: '**' can only be an entire path component
Using sample data instead...
Loaded 8 samples

Loading pre-trained BERT model...


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


BERT model loaded successfully!

=== Classification Results ===

1. Text: This movie was absolutely fantastic! Great acting and plot.
   True: positive | Predicted: negative | Confidence: 0.942
   Status: ✗

2. Text: Terrible movie, waste of time. Very boring and poorly made.
   True: negative | Predicted: negative | Confidence: 0.890
   Status: ✓

3. Text: Amazing film with excellent cinematography and soundtrack.
   True: positive | Predicted: negative | Confidence: 0.876
   Status: ✗

4. Text: Not good at all. The story was confusing and characters were weak.
   True: negative | Predicted: negative | Confidence: 0.637
   Status: ✓

5. Text: Loved every minute of it! Highly recommend to everyone.
   True: positive | Predicted: negative | Confidence: 0.952
   Status: ✗

6. Text: Awful acting and terrible script. Would not watch again.
   True: negative | Predicted: negative | Confidence: 0.852
   Status: ✓

7. Text: Outstanding performance by all actors. Brilliant direction.
   True: 